# KSchool - Master Online Data Science 

## Técnicas Avanzadas de Inteligencia Artificial

### Módulo 4: Procesamiento del Lenguaje Natural - Actividad de evaluación 2


# Estructura de la actividad

- [Descripción de la actividad](#descripcion)
- [Conjunto de datos](#datos)
- [Criterios de evaluación](#evaluacion)
- [Formato y fecha de entrega](#entrega)
- [Ejercicio 1: Algoritmo de ML Tradicional para Clasificación de Texto](#ejercicio1)
- [Ejercicio 2: Modelos de Lenguaje Pre-entrenados para Representar el Contenido Semántico del Texto](#ejercicio2)
- [Ejercicio 3: Algoritmo de DL para Clasificación de Texto](#ejercicio3)
- [Ejercicios opcionales¶](#opcionales)

<a id="descripcion"></a>
# Descripción de la actividad

En este segundo ejercicio práctico del módulo tendrás que entrenar y evaluar diferentes tipos de clasificadores de texto utilizando distintos mecanismos para representar los textos de entrada.

<a id="datos"></a>
# Conjuntos de datos

Para entrenar los distintos clasificadores de texto vamos a utilizar dos colecciones de datos públicas bastante conocidas en el mundillo:

- las [críticas de películas de IMDb](https://ai.stanford.edu/~amaas/data/sentiment/), un dataset muy habitual en tareas de análisis de opinión que vamos a utilizar para clasificación de texto binaria
- las [noticias de AG News](https://www.kaggle.com/amananandrai/ag-news-classification-dataset/), una colección de noticias organizadas en cuatro categorías muy habitual en tareas de clasificación de texto multiclase.

Ambos datasets están disponibles en multitud de sitios, pero te invito a acceder a ellos utilizando [la librería `datasets` de HuggingFace](https://github.com/huggingface/datasets). 

Para ello, asegúrate de ejecurtar la siguiente celda para instalar la librería y otra dependencia que necesitaremos más adelante:

In [ ]:
!pip install datasets

import nltk
nltk.download("stopwords")

A continuación, cargamos las dos colecciones de datos en dos objetos Python.

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")
ag_news = load_dataset("ag_news")

Estos objetos tienen forma de tabla, con filas y columnas.

In [ ]:
print(imdb)

print(f"dimensiones: {imdb.shape}")
print(f"filas: {imdb.num_rows}")
print(f"columnas: {imdb.num_columns}")
print(f"""muestras en el set de entrenamiento: {len(imdb["train"])}""")

In [ ]:
imdb["train"][:5]

Estas estructuras de datos son compatibles con otras herramientas habituales en Ciencia de Datos como `pandas`, `numpy` o `pytorch`. Tómate tu tiempo para explorar el contenido y revisa [la documentación para aprender a operar y transformar estos objetos](https://huggingface.co/docs/datasets/master/exploring.html) en otras estructuras de datos.

In [ ]:
imdb.set_format(type="pandas", columns=["text", "label"])
imdb["train"][-5:]

<a id="evaluacion"></a>
# Criterios de evaluación

Para la evaluación del ejercicio no es suficiente con proporcionar código y que este funcione. Es imprescindible proporcionar respuestas claras, justificar las decisiones tomadas, explicando pros y contras, y demostrar que se entiende en cada momento lo que se está haciendo.

Para ello, no dudes en añadir todo el texto que necesites, ya sea como comentarios de código o en celdas Markdown aparte.


| Apartado 	| Criterio de evaluación 	| Peso 	|
|:-:	|:-:	|:-:	|
| Ejercicio 1 	| la solución es completa y correcta 	| 10% 	|
| Ejercicio 2 	| la solución es completa y correcta 	| 25% 	|
| Ejercicio 3 	| la solución es completa y correcta 	| 40% 	|
| Ejercicios opcionales 	|  originalidad y claridad	| 10% 	|
| Entrega general 	| limpieza y claridad de código y respuestas 	| 15% 	|

<a id="entrega"></a>
# Formato y fecha de entrega

El cuaderno resuelto, una vez completado y listo para su entrega, deberá exportarse a formato HTML con el siguiente nombre de fichero: `apellidos-nombre-a11-mod4-2.html`

**Fecha de entrega**: TODO

<a id="ejercicio1"></a>
# Ejercicio 1: Algoritmo de ML Tradicional para Clasificación de Texto

En este primer ejercicio vamos a entrenar y evaluar un par de clasificadores de texto utilizando una estrategia basada en aprendizaje automático _tradicional_: 

1. entrenaremos un clasificador basado en _Support Vector Machine_
2. representaremos los textos de entrada como los pesos tf.idf del vocabulario
3. tokenizaremos las palabras e ignoraremos las _stop words_, es decir, palabras como preposiciones, determinantes y conjunciones que normalmente no proporcionan contenido semántico.

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support


# cargamos una lista de stop words o palabras sin contenido léxico, que querremos ignorar
stop_words = set(stopwords.words("english"))

# definimos una pipeline que combina el vectorizador y el clasificador SVM
model_sentiment = Pipeline([
                ("tfidf", TfidfVectorizer(stop_words=stop_words)),
                ("clf", LinearSVC()),
            ])

Entrenamos y evaluamos un primer clasificador de texto con el dataset de IMDb para identificar opiniones positivas y negativas.

In [ ]:
# entrenamos el modelo con el dataset de IMDb
model_sentiment.fit(imdb["train"]["text"], imdb["train"]["label"])
    
# una vez entrenado, generamos las predicciones con el test set
y_pred = model_sentiment.predict(imdb["test"]["text"])

# y calculamos las métricas de evaluación
metrics = precision_recall_fscore_support(imdb["test"]["label"], y_pred, average="macro")
print(f"Precisión: {metrics[0]}")
print(f"Cobertura: {metrics[1]}")
print(f"medida f: {metrics[2]}")

Probemos qué tal funciona con algunas frases de ejemplo.

In [ ]:
ejemplos = [
    "I really enjoyed the movie :-) It was superb!",
    "The actors and the filmaker are so bad I had to leave the movie theater before the end",
    "I love ice-cream"
]

polaridad = ["Negativa", "Positiva"]

for ejemplo in ejemplos:
  prediccion = model_sentiment.predict([ejemplo])[0]
  print(f"{ejemplo} -> {polaridad[prediccion]}")

Y ahora vamos a repetir la operacion entrenando otro clasificador con el dataset de AG News para categorizar automáticamente noticias.

In [ ]:
# definimos una pipeline que combina el vectorizador y el clasificador SVM
model_news = Pipeline([
                ("tfidf", TfidfVectorizer(stop_words=stop_words)),
                ("clf", LinearSVC()),
            ])

# entrenamos el modelo con el dataset de AG News
model_news.fit(ag_news["train"]["text"], ag_news["train"]["label"])
    
# una vez entrenado, generamos las predicciones con el test set
y_pred = model_news.predict(ag_news["test"]["text"])

# y calculamos las métricas de evaluación
metrics = precision_recall_fscore_support(ag_news["test"]["label"], y_pred, average="macro")
print(f"Precisión: {metrics[0]}")
print(f"Cobertura: {metrics[1]}")
print(f"medida f: {metrics[2]}")

Y probemos las predicciones del modelo con supuestos titulares de noticias inventados.

In [ ]:
ejemplos = [
            "Spain will face the United States in the next basketball World Championships.",
            "United Kingdom to leave the European Union by 2021, officials said.",
            "IBM to present the latest technology next Monday",
            "Inflaction and job cuts worry the stock market",
            ]

categorias = ["World", "Sports", "Business", "Sci/Tech"]
for ejemplo in ejemplos:
  prediccion = model_news.predict([ejemplo])[0]
  print(f"{ejemplo} -> {categorias[prediccion]}")

<a id="ejercicio2"></a>
# Ejercicio 2: Modelos de Lenguaje Pre-entrenados para Representar el Contenido Semántico del Texto

En este segundo ejercicio tendrás que mejorar los modelos anteriores. Para ello, entrena y evalúa un par de clasificadores de texto nuevos manteniendo el algoritmo _tradicional_ de ML pero sustituyendo el proceso de extracción de características y vectorización basado en tf.idf por alguno de los modelos de lenguaje contextual que hemos visto durante el módulo.

La idea que subyace es que cualquier modelo de lenguaje debería capturar de manera más eficaz el contenido linguístico del texto.

In [ ]:
# escribe tu código aquí


<a id="ejercicio3"></a>
# Ejercicio 3: Algoritmo de DL para Clasificación de Texto

En el tercer ejercicio tendrás que entrenar y evaluar un par de clasificadores de texto utilizando alguna de las arquitecturas de _Deep Learning_ que has aprendido durante la asignatura.

Lo que queremos comprobar ahora es si una arquitectura más sofisticada como puede ser una red neuronal profunda es capaz de mejorar el rendimiento de un clasificador de basado en _Support Vector Machines_, aunque sea a costa de añadir complejidad al proceso de entrenamiento. ¿Puedes mejorar los resultados al entrenar por ejemplo un perceptrón multicapa? ¿Puedes continuar la mejora añadiendo capas y neuronas?

A partir de este ejercicio, si vas a entrenar una red profunda, es aconsejable que entrenes los clasificadores en un entorno optimizado con aceleración por hardware. Para ello, abre el cuaderno en [Google Colab](https://colab.research.google.com/) y asegúrate de arrancar una instancia con tarjeta gráfica. Simplemente haz clic en las opciones *Entorno de ejecución* > *Cambiar tipo de entorno de ejecución* > *Acelerador por hardware GPU*.

In [ ]:
# escribe tu código aquí


<a id="opcionales"></a>
# Ejercicios opcionales

Este ejercicio es completamente abierto y puedes tratar de construir nuevos clasificadores con el objetivo de mejorar los evaluaciones obtenidas en los ejercicios anteriores.

¿Puedes mejorar los resultados al entrenar por una red neuronal de tipo CNN, LSTM o basada en Transformer? Si tienes curiosidad por saber cuáles son las soluciones que mejor resultado dan, consulta cuáles son los mejores sistemas en [la página dedicada a la clasificación de texto en _Papers with Code_](https://www.paperswithcode.com/task/text-classification).

In [ ]:
# escribe tu código aquí
